In [ ]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
import matplotlib.image as mpimg
from dlproject import get_label_mapping, get_files
slim = tf.contrib.slim

In [ ]:
checkpoint_file = './inception_resnet_v2_2016_08_30.ckpt'
label_file = './labels.txt'
train_file = 'Data/'
validation_file = 'validation/'
bottle_neck_file = 'bottle_neck/'

In [ ]:
img=mpimg.imread(train_file + '/a/0.jpg')
img.shape

In [ ]:
images = tf.placeholder(tf.float32, shape=(None, 480, 720, 3))
labels = tf.placeholder(tf.int32, shape=(None,))

# restore from the inception_resnet model
with slim.arg_scope(inception_resnet_v2_arg_scope()):
        _, end_points = inception_resnet_v2(images, num_classes = 1001, is_training = False) 

# extract the before_logit tensor from the extracted model
before_logit = end_points['PreLogitsFlatten']
    

In [ ]:
# define a placeholder for training from the bottle, not from the beginning.
bottle_neck = tf.placeholder(tf.float32, shape=(None,1536))

# now define the fine tune part of the graph
with tf.name_scope('my_fine_tune') as scope:
    batch_norm1 = tf.layers.batch_normalization(bottle_neck, name='batch_norm1')
    dropout1 = tf.layers.dropout(batch_norm1, rate=0.3, name='dropout1')
    dense1 = tf.layers.dense(dropout1, 128, activation=tf.nn.relu, name='dense1')
    batch_norm2 = tf.layers.batch_normalization(dense1, name='batch_norm2')
    dropout2 = tf.layers.dropout(batch_norm2, rate = 0.3, name='dropout2')
    logits = tf.layers.dense(dropout2, 3, name='logits')
    
    x_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
    loss = tf.reduce_mean(x_entropy, name='loss')
    correct = tf.nn.in_top_k(logits, labels, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    # extract variables that need to be trained
    weight1, bias1 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'dense1')
    weight2, bias2 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'logits')
    variables_to_train = [weight1, weight2, bias1, bias2]
    
    optimizer = tf.train.AdamOptimizer()
    train_op = slim.learning.create_train_op(loss, optimizer, variables_to_train=variables_to_train)

# defiine the name scope we don't need to restore from inception_resnet
exclude = ['InceptionResnetV2/AuxLogits', 'InceptionResnetV2/Logits',
           'my_fine_tune/','dense1','logits','batch_norm']

variables_to_restore = slim.get_variables_to_restore(exclude = exclude)
saver = tf.train.Saver(variables_to_restore)
init = tf.global_variables_initializer()

In [ ]:
# extract the data before the fine tune layer
def get_bottle_neck_data(sess, train_file, bottle_neck_file, label_file):
    id2label, label2id = get_label_mapping(label_file)
    for i in id2label:
        read_dir = train_file + i
        des_dir = bottle_neck_file + i + '/'
        os.mkdir(des_dir)
        
        files = get_files(read_dir)
        for f in files:
            img=mpimg.imread(f).reshape(1,480,720,3)
            bottle = sess.run(before_logit, feed_dict={images:img})
            np.save(des_dir + f[7:],  bottle)
            

In [ ]:
config = tf.ConfigProto()
config.allow_soft_placement = True

def train_graph():
    with tf.Session(config=config) as sess:
        sess.run(init)
        saver.restore(sess, checkpoint_file)
        
        # if there is no dir for the bottle data, then get the data first.
        if(os.path.isdir(bottle_neck_file) is not True):
            os.mkdir(bottle_neck_file)
            get_bottle_neck_data(sess, train_file, bottle_neck_file, label_file)
        
#         for i in range(10000):
#             inputs, groud_truth = *****.next_batch()
#             the_loss = sess.run([train_op], feed_dict={bottle_neck: inputs, labels: groud_truth})

In [ ]:
train_graph()